### Logarithmic probabilities are used to avoid getting multiplication of actual probabilities to zero.

In [1]:
import numpy as np

In [2]:
def fit(X_train, Y_train):
    result = {}
    class_values = set(Y_train)# level 1
    for current_class in class_values:
        result[current_class] = {}#level 2
        result["total_data"] = len(Y_train)
        current_class_rows = (Y_train == current_class)
        X_train_current = X_train[current_class_rows]
        Y_train_current = Y_train[current_class_rows]
        n_features = X_train.shape[1]
        result[current_class]["total_count"] = len(Y_train_current)
        for j in range(1, n_features + 1):
            #j_1 = j - 1
            result[current_class][j] = {}# level 3
            all_possible_values = set(X_train[:, j - 1])
            for current_value in all_possible_values:
                result[current_class][j][current_value] = (X_train_current[:, j - 1] == current_value).sum()
    return result

In [44]:
def probability(dictionary, x, current_class):# log probability funtion
    #p(y=current_class)
    output = np.log(dictionary[current_class]["total_count"]) - np.log(dictionary["total_data"])
    n_features = len(dictionary[current_class].keys()) - 1
    # -1 for extra feature total_count
    for j in range(1, n_features + 1):
        xj = x[j - 1]
        count_current_class_with_value_xj = dictionary[current_class][j][xj] + 1
        # +1 for laplace correction
        count_current_class = dictionary[current_class]["total_count"] + len(dictionary[current_class][j].keys())
        current_xj_prob = np.log(count_current_class_with_value_xj) - np.log(count_current_class)
        output = output + current_xj_prob
    return output

In [45]:
def predictSinglePoint(dictionary, x):
    classes = dictionary.keys()
    best_p = -1000
    best_class = -1
    first_run = True
    for current_class in classes:
        if (current_class == "total_data"):
            continue
        p_current_class = probability(dictionary, x, current_class)#probability of current class
        if (first_run or p_current_class > best_p):
            best_p = p_current_class
            best_class = current_class
            
        first_run = False
    return best_class

In [46]:
def predict(dictionary, X_test):
    y_pred = []
    for x in X_test:
        x_class = predictSinglePoint(dictionary, x)
        y_pred.append(x_class)
    return y_pred

In [47]:
def make_labelled(column):#lebels contineous data
    second_limit = column.mean()
    first_limit = second_limit / 2
    third_limit = second_limit * 3 / 2
    
    for i in range(len(column)):
        if(column[i] < first_limit):#1st label is 0
            column[i] = 0
        elif(column[i] < second_limit):#2nd label is 1
            column[i] = 1
        elif(column[i] < third_limit):#3rd label is 2
            column[i] = 2
        else:
            column[i] = 3
    return column

In [48]:
from sklearn import datasets
iris = datasets.load_iris()
X = iris.data
Y = iris.target

In [49]:
for i in range(X.shape[-1]):
    X[:, i] = make_labelled(X[:, i])

In [50]:
X

array([[1., 2., 0., 0.],
       [1., 1., 0., 0.],
       [1., 2., 0., 0.],
       [1., 2., 0., 0.],
       [1., 2., 0., 0.],
       [1., 2., 0., 0.],
       [1., 2., 0., 0.],
       [1., 2., 0., 0.],
       [1., 1., 0., 0.],
       [1., 2., 0., 0.],
       [1., 2., 0., 0.],
       [1., 2., 0., 0.],
       [1., 1., 0., 0.],
       [1., 1., 0., 0.],
       [1., 2., 0., 0.],
       [1., 2., 0., 0.],
       [1., 2., 0., 0.],
       [1., 2., 0., 0.],
       [1., 2., 0., 0.],
       [1., 2., 0., 0.],
       [1., 2., 0., 0.],
       [1., 2., 0., 0.],
       [1., 2., 0., 0.],
       [1., 2., 0., 0.],
       [1., 2., 1., 0.],
       [1., 1., 0., 0.],
       [1., 2., 0., 0.],
       [1., 2., 0., 0.],
       [1., 2., 0., 0.],
       [1., 2., 0., 0.],
       [1., 2., 0., 0.],
       [1., 2., 0., 0.],
       [1., 2., 0., 0.],
       [1., 2., 0., 0.],
       [1., 2., 0., 0.],
       [1., 2., 0., 0.],
       [1., 2., 0., 0.],
       [1., 2., 0., 0.],
       [1., 1., 0., 0.],
       [1., 2., 0., 0.],


In [51]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.25, random_state = 0)
X_test.shape

(38, 4)

In [52]:
dictionary = fit(X_train, Y_train)

In [53]:
Y_pred = predict(dictionary, X_test)

In [57]:
from sklearn.metrics import classification_report,confusion_matrix
print(classification_report(Y_test, Y_pred))
print((confusion_matrix(Y_test, Y_pred)))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        13
           1       0.94      1.00      0.97        16
           2       1.00      0.89      0.94         9

    accuracy                           0.97        38
   macro avg       0.98      0.96      0.97        38
weighted avg       0.98      0.97      0.97        38

[[13  0  0]
 [ 0 16  0]
 [ 0  1  8]]


In [60]:
from sklearn.naive_bayes import GaussianNB
clf = GaussianNB()
clf.fit(X_train, Y_train)
Y_pred = clf.predict(X_test)
print(classification_report(Y_test, Y_pred))
print(confusion_matrix(Y_test, Y_pred))

              precision    recall  f1-score   support

           0       1.00      0.85      0.92        13
           1       0.76      1.00      0.86        16
           2       1.00      0.67      0.80         9

    accuracy                           0.87        38
   macro avg       0.92      0.84      0.86        38
weighted avg       0.90      0.87      0.87        38

[[11  2  0]
 [ 0 16  0]
 [ 0  3  6]]


In [61]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB()
clf.fit(X_train, Y_train)
Y_pred = clf.predict(X_test)
print(classification_report(Y_test, Y_pred))
print(confusion_matrix(Y_test, Y_pred))

              precision    recall  f1-score   support

           0       1.00      0.85      0.92        13
           1       0.00      0.00      0.00        16
           2       0.36      1.00      0.53         9

    accuracy                           0.53        38
   macro avg       0.45      0.62      0.48        38
weighted avg       0.43      0.53      0.44        38

[[11  2  0]
 [ 0  0 16]
 [ 0  0  9]]
